In [ ]:
# Requiremnts
%pip install selenium
%pip install bs4
%pip install pandas

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [ ]:
def get_source(url):
    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument('log-level=3')
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument('--blink-settings=imagesEnabled=false')
    chrome_options.add_argument('--disable-images')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    # Initialize the WebDriver
    driver = webdriver.Chrome(options=chrome_options)

    # URL of the page you want to scrape
    url = url
    driver.get(url)
    html = ""  
    try:
        # Wait until all "read more" buttons are present
        wait = WebDriverWait(driver, 10)
        buttons = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//span[text()=' read more']")))


        # Click each button
        for i, button in enumerate(buttons):
            try:
                # Scroll into view
                print(f"\rExpanding all the {i+1}/{len(buttons)} descriptions...", end="")
                driver.execute_script("arguments[0].scrollIntoView();", button)
                sleep(0.3)

                # Ensure the element is clickable
                wait.until(EC.element_to_be_clickable(button))

                # Click using JavaScript if regular click fails
                driver.execute_script("arguments[0].click();", button)

                # sleep(0.3)
            except Exception as e:
                print(f"Error clicking button: {e}")


        # Get page source after clicking all buttons
        html = driver.page_source if driver.page_source else "" 

    except Exception as e:
        print(f"Error: {e}")


    # Save the page source to a file
    with open('page_source.html', 'w', encoding='utf-8') as file:
        file.write(html)

    # Close the driver
    driver.quit()

In [ ]:
# url = "https://www.zomato.com/mumbai/tru-falafel-khar/order"
url = "https://www.zomato.com/mumbai/nothing-but-chicken-pali-hill-bandra-west/order"
# url = "https://www.zomato.com/mumbai/dadar-social-dadar-west/order"

get_source(url)

In [ ]:
with open(r"page_source.html",encoding='utf-8') as html:
    content = html.read()

In [ ]:
soup = BeautifulSoup(content,'html.parser')

In [ ]:
soup

In [ ]:
food_items = soup.find_all('div', {'type': ['veg', 'non-veg']})
data = []  
for elm in food_items:
    # Determine the food type
    food_type = elm.get('type')

    # Find the sibling element
    sibling = elm.find_parent().find_next_sibling()

    if not sibling:
        sibling = elm.find_next_sibling()

    if sibling:
        # Find the section tag by moving up the hierarchy from the sibling
        section_tag = sibling.find_parent()
        while section_tag and section_tag.name != 'section':
            section_tag = section_tag.find_parent()

        if section_tag:
            # Extract the h4 tag text within the section
            section = section_tag.find('h4')
            if section:
                section_text = section.get_text().strip()
            else:
                print("No section found within the section.")
        else:
            print("No section tag found.")

        # Extract the sub headding
        sub_heading = ""
        if section_tag:
            sub_hedding_find = section_tag.select('section > div > p')
            if sub_heading:            
                for p_tag in sub_hedding_find:
                    sub_heading = p_tag.get_text().strip()
            else:
                sub_heading = "No sub headding"

        # Extract the title
        title_h = sibling.select_one("div > div > div > h4")
        title = title_h.get_text().strip() if title_h else None

        # Extract the votes
        vote_span = sibling.select_one("span:contains('votes')")
        vote = vote_span.get_text().strip() if vote_span else None

        # Extract the price
        price_span = sibling.select_one("div > div > div > div > span:not(:contains('votes'))")
        price = price_span.get_text().replace('₹', '').strip() if price_span else None

        # Extract the description
        description_p = sibling.select_one("p")
        description = description_p.get_text().strip() if description_p else None

        data.append({
            'Title': title,
            'Type': food_type,
            'Price': price,
            'Votes': vote,
            'Description': description,
            'Sub Headding': sub_heading,
            'Section': section_text
        })

        # print(f"Title: {title}\nType: {food_type}\nPrice: {price}\nVotes: {vote}\nDescription: {description}\n")

In [ ]:
df = pd.DataFrame(data)

try:
    df.to_csv('output.csv', index=False)
    print("Output saved to output.csv")
except Exception as e:
    print(e)